## 瑜珈角度製作

為了要製作瑜珈評分系統，我們需要各個瑜珈動作關節角度的平均值、標準差

- 輸入: 有圖片的資料夾路徑
- 輸出:
    - JsonFile/pose/sample_score.json: 給予分數計算系統使用，平均關節點角度
    - JsonFile/pose/std_angle.json:  關節點角度標準差
    - JsonFile/pose/weight.json: 關節點的計分權重

- 詳細介紹可參考[論文](https://docs.google.com/document/d/1sWPlbKvfi4x-Idih0DI4toHYrIhyDv6y/edit?usp=sharing&ouid=114571548892193624282&rtpof=true&sd=true) : 3.2. 自動瑜珈墊評分系統
  
### 流程
- 先準備各個動作的資料集到 first_input
- 先第一次執行角度測試，會輸出圖片偵測的關節點，篩選關節點偵測好的到 second_input
- 執行第二次執行角度測試，取得最終的關節點角度



In [17]:
from yogaPoseDetect import YogaPose
import cv2
from pathlib import Path
import numpy as np
import toolkit

from AngleCalculator import calculate_angle
from develop_tool import save_json, get_Mediapipe_point, get_image_files, draw_skeleton

json_path = "../JsonFile"

# 計算並儲存角度
def calculate_angles_record(angles_data, pose_name):
    keypoints = list(angles_data[0].keys())
    average_angle = dict()
    std_angle = dict()

    for keypoint in keypoints:
        values = [entry[keypoint] for entry in angles_data]
        mean_value = np.mean(values)
        std_value = np.std(values)
        average_angle[keypoint] = round(mean_value, 1)
        std_angle[keypoint] = round(std_value, 1)
        # print(f"{keypoint} 平均值和标准差: {mean_value}, {std_value}")
    # 
    # print("avg", average_angle)
    # print("std", std_angle)
    save_json(json_path + "/" + pose_name, "sample.json", average_angle)
    save_json(json_path + "/" + pose_name, "std_angle.json", std_angle)
    
    
def create_image_grid(images, image_names):
    # 設置合成圖片的寬度和高度
    width = 1500
    height = 900
    # 設置每個小格子的寬度和高度
    cell_width = width // 5
    cell_height = height // 3 
    # 設置文字的字型和大小
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    font_color = (0, 0, 0)

    # 創建一個新的白色背景圖片
    new_image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    idx = 0
    while idx < len(images):
        for i in range(3):
            for j in range(5):
                if idx >= len(images):
                    break
                # 讀取圖片
                img = images[idx]
                # 調整圖片大小以符合格子大小
                img = cv2.resize(img, (cell_width, cell_height))
                # 計算圖片在合成圖片中的位置
                x = j * cell_width
                y = i * cell_height
                # 將圖片貼到合成圖片中的正確位置
                new_image[y:y+cell_height, x:x+cell_width] = img
                # 寫入圖片名稱
                cv2.putText(new_image, image_names[idx], (x, y + cell_height - 10), font, font_scale, font_color, font_thickness)
                idx += 1
    
        # 儲存合成圖片
        
        cv2.imshow(f"output_{idx // 12}.jpg", new_image)

        new_image = np.ones((height, width, 3), dtype=np.uint8) * 255
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
            
# 進行圖片轉換
def process_images(input_path,  pose_name):
    print(input_path)
    image_files = get_image_files(input_path / pose_name)
    print(input_path / pose_name)
    print(image_files)
    angles_data = []


    output_images = []
    image_names = []
    for image_file in image_files:
        print(image_file)
        image = cv2.imread(image_file, cv2.IMREAD_COLOR)
        point2d, point3d = get_Mediapipe_point(image)
        
        if point2d is not None:
            # 計算角度
            angle = calculate_angle(pose_name, point3d)
            angles_data.append(dict(angle))
            print(angle)
            
            # 將角度畫在放進圖片中
            output_image = draw_skeleton(image,  point2d)
            output_name = Path(image_file).name
            output_images.append(output_image)
            image_names.append(output_name)

    create_image_grid(output_images, image_names)
    # 計算並儲存角度的標準差
    
    print(angles_data)
    calculate_angles_record(angles_data, pose_name)

In [21]:
## 路徑
#  ["Tree Style", "Warrior2 Style", "Plank", "Reverse Plank", "Child's pose", "Downward dog", "Low Lunge", "Seated Forward Bend", "Bridge pose", "Pyramid pose"]
pose_name = 'Tree Style'

dataset_path = Path("C:/Users/Cobra/Desktop/Code Project/yoga project/DataSet")

In [22]:
## 第一次測試圖片
input_path = dataset_path / "first_input" 

process_images(input_path, pose_name)

C:\Users\Cobra\Desktop\Code Project\yoga project\DataSet\first_input
C:\Users\Cobra\Desktop\Code Project\yoga project\DataSet\first_input\Tree Style
['C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\0-1.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\1-0.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\11-0.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\13-0.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\14-0.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\15-0.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\16-0.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\first_input\\Tree Style\\16-1.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yo

In [16]:
# 篩選圖片後，測量角度
input_path = dataset_path / "second_input" 

# print(input_path)
process_images(input_path, pose_name)

C:\Users\Cobra\Desktop\Code Project\yoga project\DataSet\second_input
C:\Users\Cobra\Desktop\Code Project\yoga project\DataSet\second_input\Downward dog
['C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\second_input\\Downward dog\\24. dsc_1007-resized-600.jpg', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\second_input\\Downward dog\\1. 1.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\second_input\\Downward dog\\1. 5-benefits-of-downward-facing-dog-pose.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\second_input\\Downward dog\\10. screen-shot-2017-09-15-at-17.00.06-1024x585.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\second_input\\Downward dog\\11. yoga_anatomy_using_muscle_awareness_to_lower_your_heels_in_downward_facing_dog_pose.png', 'C:\\Users\\Cobra\\Desktop\\Code Project\\yoga project\\DataSet\\second_input\\Downward dog\\12. 66a62b8c606fd88e0401b5af0a7cbca7.png', 'C:\\Us